## Sissejuhatus

Selles õppetükis käsitletakse:  
- Mis on funktsiooni kutsumine ja selle kasutusjuhud  
- Kuidas luua funktsiooni kutset Azure OpenAI abil  
- Kuidas integreerida funktsiooni kutse rakendusse  

## Õpieesmärgid

Pärast selle õppetüki läbimist oskad ja mõistad:  

- Funktsiooni kutsumise eesmärki  
- Funktsiooni kutse seadistamist Azure OpenAI teenuse abil  
- Tõhusate funktsioonikutsungite kujundamist oma rakenduse kasutusjuhtumi jaoks  


## Funktsioonikutsete mõistmine

Selles õppetükis soovime luua funktsiooni meie haridusettevõtte jaoks, mis võimaldab kasutajatel kasutada vestlusboti tehniliste kursuste leidmiseks. Soovitame kursusi, mis sobivad nende oskuste tasemele, praegusele rollile ja huvipakkuvale tehnoloogiale.

Selle saavutamiseks kasutame kombinatsiooni:
 - `Azure Open AI`, et luua kasutajale vestluskogemus
 - `Microsoft Learn Catalog API`, et aidata kasutajatel leida kursusi vastavalt nende päringule
 - `Function Calling`, et võtta kasutaja päring ja saata see funktsioonile API päringu tegemiseks.

Alustuseks vaatame, miks me üldse tahaksime kasutada funktsioonikutseid:

print("Sõnumid järgmises päringus:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # saada GPT-lt uus vastus, kus see näeb funktsiooni vastust


print(second_response.choices[0].message)


### Miks kasutada funktsioonide kutsumist

Kui oled läbinud mõne teise õppetunni selles kursuses, siis tõenäoliselt mõistad juba suurte keelemudelite (LLM-ide) kasutamise võimsust. Loodetavasti oled märganud ka nende piiranguid.

Funktsioonide kutsumine on Azure Open AI teenuse funktsioon, mis aitab ületada järgmisi piiranguid:
1) Järjepidev vastuse formaat  
2) Võimalus kasutada rakenduse teistest allikatest pärit andmeid vestluskontekstis  

Enne funktsioonide kutsumist olid LLM-i vastused struktureerimata ja ebajärjekindlad. Arendajad pidid kirjutama keerulist valideerimiskoodi, et tagada iga vastuse variatsiooni käsitlemine.

Kasutajad ei saanud vastuseid küsimustele nagu "Milline on praegune ilm Stockholmis?". Seda seetõttu, et mudelid olid piiratud ajaga, mil andmed treeniti.

Vaatame allpool olevat näidet, mis illustreerib seda probleemi:

Oletame, et soovime luua andmebaasi õpilaste andmetega, et saaksime neile soovitada sobivaid kursusi. Allpool on kaks õpilaste kirjeldust, mis sisaldavad väga sarnaseid andmeid.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Soovime selle saata LLM-ile, et andmeid analüüsida. Seda saab hiljem kasutada meie rakenduses, et saata API-le või salvestada andmebaasi.

Loome kaks identset juhist, milles anname LLM-ile teada, millist teavet meid huvitab:


Me soovime selle saata LLM-ile, et analüüsida osasid, mis on meie toote jaoks olulised. Nii saame luua kaks identset käsku, et juhendada LLM-i:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Pärast nende kahe viipa loomist saadame need LLM-ile, kasutades `openai.ChatCompletion`. Me salvestame viiba muutujasse `messages` ja määrame rolliks `user`. See on mõeldud matkima kasutaja sõnumit, mis kirjutatakse vestlusrobotile.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Nüüd saame saata mõlemad päringud LLM-ile ja uurida saadud vastust.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Kuigi juhised on samad ja kirjeldused sarnased, võime saada `Grades` omaduse erinevaid vorminguid.

Kui käivitada ülaltoodud lahter mitu korda, võib vorming olla kas `3.7` või `3.7 GPA`.

See juhtub seetõttu, et LLM võtab sisendiks struktureerimata andmeid kirjutatud juhise kujul ja tagastab samuti struktureerimata andmeid. Meil on vaja struktureeritud vormingut, et teaksime, mida oodata, kui neid andmeid salvestame või kasutame.

Funktsionaalse väljakutse kasutamise abil saame tagada, et saame tagasi struktureeritud andmeid. Funktsionaalse väljakutse kasutamisel LLM tegelikult ei kutsu ega käivita ühtegi funktsiooni. Selle asemel loome LLM-ile struktuuri, mida vastustes järgida. Seejärel kasutame neid struktureeritud vastuseid, et teada, millist funktsiooni oma rakendustes käivitada.


![Funktsiooni kutsumise vooskeem](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.et.png)


Seejärel saame funktsioonist tagastatud väärtuse võtta ja selle LLM-ile tagasi saata. LLM vastab seejärel loomulikus keeles, et vastata kasutaja päringule.


### Funktsioonikõnede kasutusjuhtumid

**Väliste tööriistade kasutamine**  
Vestlusrobotid on suurepärased kasutajate küsimustele vastuste andmiseks. Funktsioonikõnede abil saavad vestlusrobotid kasutada kasutajate sõnumeid teatud ülesannete täitmiseks. Näiteks võib õpilane paluda vestlusrobotil: "Saada e-kiri minu juhendajale, öeldes, et vajan selle teemaga rohkem abi." See võib teha funktsioonikõne `send_email(to: string, body: string)`.

**API või andmebaasi päringute loomine**  
Kasutajad saavad leida teavet, kasutades loomulikku keelt, mis muudetakse vormindatud päringuks või API päringuks. Näiteks võib õpetaja küsida: "Kes on õpilased, kes lõpetasid viimase ülesande," mis võib kutsuda funktsiooni nimega `get_completed(student_name: string, assignment: int, current_status: string)`.

**Struktureeritud andmete loomine**  
Kasutajad saavad võtta tekstiploki või CSV-faili ja kasutada LLM-i, et sellest olulist teavet välja võtta. Näiteks võib õpilane muuta Wikipedia artikli rahulepingutest AI-mälukaartide loomiseks. Seda saab teha funktsiooni abil `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Esimese funktsioonikõne loomine

Funktsioonikõne loomise protsess hõlmab kolme peamist sammu:  
1. Chat Completions API kutsumine koos funktsioonide loendi ja kasutaja sõnumiga  
2. Mudeli vastuse lugemine, et sooritada tegevus, näiteks funktsiooni või API-kõne täitmine  
3. Uue kõne tegemine Chat Completions API-le, kasutades funktsiooni vastust, et luua kasutajale vastus.  


![Funktsiooni kutse voog](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.et.png)


### Funktsiooni väljakutse elemendid

#### Kasutaja sisend

Esimene samm on luua kasutaja sõnum. See võib olla dünaamiliselt määratud, võttes väärtuse tekstisisendist, või saate siin väärtuse määrata. Kui see on teie esimene kord töötada Chat Completions API-ga, peame määratlema sõnumi `role` ja `content`.

`Role` võib olla kas `system` (reeglite loomine), `assistant` (mudel) või `user` (lõppkasutaja). Funktsiooni väljakutse jaoks määrame selle `user`-iks ja lisame näitena küsimuse.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktsioonide loomine.

Järgmisena määratleme funktsiooni ja selle funktsiooni parameetrid. Siin kasutame ainult ühte funktsiooni nimega `search_courses`, kuid võite luua ka mitu funktsiooni.

**Oluline**: Funktsioonid lisatakse süsteemisõnumisse LLM-ile ja need arvestatakse saadaval olevate tokenite hulka.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitsioonid**

`name` - Funktsiooni nimi, mida soovime kutsuda.

`description` - Kirjeldus, kuidas funktsioon töötab. Siin on oluline olla konkreetne ja selge.

`parameters` - Väärtuste ja vormingu loetelu, mida soovite, et mudel oma vastuses genereeriks.

`type` - Andmetüüp, milles omadused salvestatakse.

`properties` - Spetsiifiliste väärtuste loetelu, mida mudel oma vastuses kasutab.

`name` - Omaduse nimi, mida mudel kasutab oma vormindatud vastuses.

`type` - Selle omaduse andmetüüp.

`description` - Konkreetse omaduse kirjeldus.

**Valikuline**

`required` - Nõutav omadus, et funktsiooni kutse saaks lõpule viia.


### Funktsiooni väljakutse tegemine
Pärast funktsiooni defineerimist peame selle nüüd lisama Chat Completion API väljakutsesse. Selleks lisame päringusse `functions`. Antud juhul on see `functions=functions`.

Samuti on võimalus määrata `function_call` väärtuseks `auto`. See tähendab, et laseme LLM-il otsustada, millist funktsiooni tuleks kasutaja sõnumi põhjal kutsuda, selle asemel et ise määrata.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nüüd vaatame vastust ja selle vormingut:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Nagu näha, on funktsiooni nimi välja toodud ja kasutaja sõnumi põhjal suutis LLM leida andmed, mis sobivad funktsiooni argumentideks.


## 3. Funktsioonikutsete integreerimine rakendusse

Pärast seda, kui oleme testinud LLM-i vormindatud vastust, saame selle nüüd rakendusse integreerida.

### Voogude haldamine

Selle rakendusse integreerimiseks järgime järgmisi samme:

Esmalt teeme OpenAI teenustele päringu ja salvestame sõnumi muutujasse nimega `response_message`.


In [ ]:
response_message = response.choices[0].message

Nüüd defineerime funktsiooni, mis kutsub Microsoft Learn API-d, et saada kursuste nimekiri:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Parima tava kohaselt vaatame esmalt, kas mudel soovib funktsiooni kutsuda. Seejärel loome ühe saadaolevatest funktsioonidest ja seome selle kutsutava funktsiooniga.  
Seejärel võtame funktsiooni argumendid ja kaardistame need LLM-i argumentidega.  

Lõpuks lisame funktsiooni kutse sõnumi ja väärtused, mille tagastas `search_courses` sõnum. See annab LLM-ile kogu vajaliku teabe, et vastata kasutajale loomulikus keeles.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nüüd saadame uuendatud sõnumi LLM-ile, et saada loomuliku keele vastus API JSON-vormingus vastuse asemel.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Koodi väljakutse

Suurepärane töö! Azure Open AI funktsioonikutsumise õppimise jätkamiseks saate luua: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- Rohkem funktsiooni parameetreid, mis aitaksid õppijatel leida rohkem kursusi. Saadaval olevad API parameetrid leiate siit:  
- Looge teine funktsioonikutsumine, mis võtab õppijalt rohkem teavet, näiteks tema emakeele  
- Looge veakäsitlus juhuks, kui funktsioonikutsumine ja/või API-kutsumine ei tagasta sobivaid kursusi  



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
